# PyTorch Introduction

PyTorch is not just a Deep Learning research platform, it's also a replacement for NumPy to use the power of GPUs. One of Pytorch's advantages is the use of [autograd](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html): Automatic Differentiation. The autograd package provides automatic differentiation for all operations on Tensors, which make PyTorch faster and more flexible than Keras.


## Cheat Sheet

### Deep Learning
#### Model

In [ ]:
nn.Linear(m,n)                                # fully connected layer from
                                              # m to n units

nn.ConvXd(m,n,s)                              # X dimensional conv layer from
                                              # m to n channels where X⍷{1,2,3}
                                              # and the kernel size is s

nn.MaxPoolXd(s)                               # X dimension pooling layer
                                              # (notation as above)

nn.BatchNorm                                  # batch norm layer
nn.RNN/LSTM/GRU                               # recurrent layers
nn.Dropout(p=0.5, inplace=False)              # dropout layer for any dimensional input
nn.Dropout2d(p=0.5, inplace=False)            # 2-dimensional channel-wise dropout
nn.Embedding(num_embeddings, embedding_dim)   # (tensor-wise) mapping from
                                              # indices to embedding vectors

#### Loss Funtions

In [ ]:
nn.X                                  # where X is BCELoss, CrossEntropyLoss,
                                      # L1Loss, MSELoss, NLLLoss, SoftMarginLoss,
                                      # MultiLabelSoftMarginLoss, CosineEmbeddingLoss,
                                      # KLDivLoss, MarginRankingLoss, HingeEmbeddingLoss
                                      # or CosineEmbeddingLoss

#### Activation Functions

In [ ]:
nn.X                                  # where X is ReLU, ReLU6, ELU, SELU, PReLU, LeakyReLU,
                                      # Threshold, HardTanh, Sigmoid, Tanh,
                                      # LogSigmoid, Softplus, SoftShrink,
                                      # Softsign, TanhShrink, Softmin, Softmax,
                                      # Softmax2d or LogSoftmax

#### Optimizers

In [ ]:
opt = optim.x(model.parameters(), ...)      # create optimizer
opt.step()                                  # update weights
optim.X                                     # where X is SGD, Adadelta, Adagrad, Adam,
                                            # SparseAdam, Adamax, ASGD,
                                            # LBFGS, RMSProp or Rprop

#### Learning Rate

In [ ]:
scheduler = optim.X(optimizer,...)      # create lr scheduler
scheduler.step()                        # update lr at start of epoch
optim.lr_scheduler.X                    # where X is LambdaLR, StepLR, MultiStepLR,
                                        # ExponentialLR or ReduceLROnPLateau

## Datasets

In [ ]:
Dataset                    # abstract class representing dataset
TensorDataset              # labelled dataset in the form of tensors
Concat Dataset             # concatenation of Datasets

DataLoader(dataset, batch_size=1, ...)      # loads data batches agnostic
                                            # of structure of individual data points

sampler.Sampler(dataset,...)                # abstract class dealing with
                                            # ways to sample from dataset

sampler.XSampler where ...                  # Sequential, Random, Subset,
                                            # WeightedRandom or Distributed

## PyTorch vs Keras

## MNIST Training PyTorch

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


Extracting ../../data\MNIST\raw\train-images-idx3-ubyte.gz to ../../data\MNIST\raw



Extracting ../../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../../data\MNIST\raw



Extracting ../../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data\MNIST\raw



Extracting ../../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data\MNIST\raw
Processing...
Done!
Epoch [1/5], Step [100/600], Loss: 0.4749
Epoch [1/5], Step [200/600], Loss: 0.1614
Epoch [1/5], Step [300/600], Loss: 0.2061
Epoch [1/5], Step [400/600], Loss: 0.2086
Epoch [1/5], Step [500/600], Loss: 0.2187
Epoch [1/5], Step [600/600], Loss: 0.1495
Epoch [2/5], Step [100/600], Loss: 0.1294
Epoch [2/5], Step [200/600], Loss: 0.0783
Epoch [2/5], Step [300/600], Loss: 0.2041
Epoch [2/5], Step [400/600], Loss: 0.1390
Epoch [2/5], Step [500/600], Loss: 0.1579
Epoch [2/5], Step [600/600], Loss: 0.1023
Epoch [3/5], Step [100/600], Loss: 0.0793
Epoch [3/5], Step [200/600], Loss: 0.0418
Epoch [3/5], Step [300/600], Loss: 0.0579
Epoch [3/5], Step [400/600], Loss: 0.0694
Epoch [3/5], Step [500/600], Loss: 0.0954
Epoch [3/5], Step [600/600], Loss: 0.0497
Epoch [4/5], Step [100/600], Loss: 0.0723
Epoch [4/5], Step [200/600], Loss: 0.0464
Epoch [4/5], Step [300/600], Loss: 0.0729
Epoch [4/5], St

## MNIST Training Keras

In [13]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - ETA: 2:07 - loss: 2.3227 - accuracy: 0.10 - ETA: 1:24 - loss: 2.2800 - accuracy: 0.15 - ETA: 1:11 - loss: 2.2357 - accuracy: 0.20 - ETA: 1:04 - loss: 2.1764 - accuracy: 0.24 - ETA: 1:00 - loss: 2.1714 - accuracy: 0.23 - ETA: 57s - loss: 2.1473 - accuracy: 0.2513 - ETA: 55s - loss: 2.0902 - accuracy: 0.298 - ETA: 53s - loss: 2.0020 - accuracy: 0.339 - ETA: 52s - loss: 1.9538 - accuracy: 0.359 - ETA: 51s - loss: 1.9352 - accuracy: 0.370 - ETA: 50s - loss: 1.8785 - accuracy: 0.392 - ETA: 49s - loss: 1.8246 - accuracy: 0.404 - ETA: 49s - loss: 1.7757 - accuracy: 0.417 - ETA: 48s - loss: 1.7382 - accuracy: 0.430 - ETA: 48s - loss: 1.7007 - accuracy: 0.442 - ETA: 47s - loss: 1.6623 - accuracy: 0.458 - ETA: 47s - loss: 1.6164 - accuracy: 0.474 - ETA: 46s - loss: 1.5786 - accuracy: 0.486 - ETA: 46s - loss: 1.5436 - 

#### Results

We trained the PyTorch model for less time and achived 97.86 % and a accuracy of 99.26 % in Keras showing that we can still achieve great accuracy in the higher lever framework.